In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

## Deploying the Model into Production with Merlin Systems and Triton IS

At this point, when you reach out to this notebook, we expect that you have already executed the first notebook `01-Building-Recommender-Systems-PoC.ipynb` and exported all the required files and models. 



### Import required libraries and functions

In [2]:
from nvtabular import ColumnSchema, Schema

from merlin.systems.dag.ensemble import Ensemble
from merlin.systems.dag.ops.session_filter import FilterCandidates
from merlin.systems.dag.ops.softmax_sampling import SoftmaxSampling
from merlin.systems.dag.ops.tensorflow import PredictTensorflow
from merlin.systems.dag.ops.unroll_features import UnrollFeatures

from merlin.systems.triton.utils import run_triton_server, run_ensemble_on_tritonserver

/nvtabular/nvtabular/graph.py:23: FutureWarning: The `nvtabular.graph` module has moved to `merlin.dag`. Support for importing from `nvtabular.graph` is deprecated, and will be removed in a future version. Please update your imports to import from `merlin.dag`.
  warnings.warn(
/nvtabular/nvtabular/io.py:23: FutureWarning: The `nvtabular.io` module has moved to `merlin.io`. Support for importing from `nvtabular.io` is deprecated, and will be removed in a future version. Please update your imports to import from `merlin.io`.
  warnings.warn(
/nvtabular/nvtabular/utils.py:23: FutureWarning: The `nvtabular.utils` module has moved to `merlin.core.utils`. Support for importing from `nvtabular.utils` is deprecated, and will be removed in a future version. Please update your imports to import from `merlin.core.utils`.
  warnings.warn(
/nvtabular/nvtabular/dispatch.py:23: FutureWarning: The `nvtabular.dispatch` module has moved to `merlin.core.dispatch`. Support for importing from `nvtabular.d

### Feast Apply and Materialize

We have defined our user and item features definitions in the `user_features.py` and  `item_features.py` files. With FeatureView() users can register data sources in their organizations into Feast, and then use those data sources for both training and online inference. In the `user_features.py` and `item_features.py` files, we are telling Feast where to find user and item features.


Before we move on to the next steps, we need to perform `apply` and `materizalize` commands as directed below:

```
# open a terminal and navigate to the `feature_repo` folder

cd /Merlin/examples/PoC/feature_repo

# run the following commands

feast apply
feast materialize 1995-01-01T01:01:01 2025-01-01T01:01:01
```

With `feast apply` we can apply the changes to create our feature registry and store all entity and feature view definitions in a local file called `registry.db`.

When you run the `feast materialize ..` command you will see a print out message <i>Materializing 2 feature views from 1995-01-01 01:01:01+00:00 to 2025-01-01 01:01:01+00:00 into the sqlite online store </i> on your terminal. [materialization](https://docs.feast.dev/how-to-guides/running-feast-in-production) operation is done to keep our online store up to date. For that we need to run a job that loads feature data from our feature view sources into our online store. As we add new features to our offline stores, we can continuously materialize them to keep our online store fresh. 

Note that materialization step takes some time.. 

Now, let's check our feature_repo structure again after we ran `apply` and `materialize` commands.

In [3]:
!tree ./feature_repo

./feature_repo
├── __init__.py
├── data
│   ├── item_features.parquet
│   ├── online_store.db
│   ├── registry.db
│   └── user_features.parquet
├── feature_store.yaml
├── item_features.py
└── user_features.py

1 directory, 8 files


In [4]:
# import sys
# sys.path.append("/systems/")

### Steps
- 
-
-

In [5]:
from nvtabular.loader.tf_utils import configure_tensorflow, get_dataset_schema_from_feature_columns

configure_tensorflow()

<function tensorflow.python.dlpack.dlpack.from_dlpack(dlcapsule)>

In [7]:
import os
os.makedirs("tmp")

In [8]:
base_path = "/Merlin/examples/PoC/"
faiss_index_path = base_path + 'tmp' + "/index.faiss"
feast_repo_path = base_path + "feature_repo/"
retrieval_model_path = base_path + "query_tower/"
ranking_model_path = base_path + "dlrm/"

To help you visualize what we will be doing, here’s what our feature_repo structure look like:

In [9]:
faiss_index_path, feast_repo_path, retrieval_model_path

('/Merlin/examples/PoC/tmp/index.faiss',
 '/Merlin/examples/PoC/feature_repo/',
 '/Merlin/examples/PoC/query_tower/')

In [10]:
import numpy as np
import cudf
import feast
import faiss
import pandas as pd

from merlin.systems.dag.ops.faiss import QueryFaiss, setup_faiss 
from merlin.systems.dag.ops.feast import QueryFeast 


request_schema = Schema(
    [
        ColumnSchema("user_id", dtype=np.int32),
    ]
)

item_embeddings = np.ascontiguousarray(
    pd.read_parquet(base_path + "item_embeddings.parquet").to_numpy()
)

feature_store = feast.FeatureStore(feast_repo_path)
setup_faiss(item_embeddings, faiss_index_path)

user_features = ["user_id"] >> QueryFeast.from_feature_view(
    store=feature_store,
    path=feast_repo_path,
    view="user_features",
    column="user_id",
    include_id=True,
)

/usr/local/lib/python3.8/dist-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/usr/local/lib/python3.8/dist-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
03/30/2022 12:40:14 AM INFO:Loading faiss with AVX2 support.
03/30/2022 12:40:14 AM INFO:Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
03/30/2022 12:40:14 AM INFO:Loading faiss.
03/30/2022 12:40:14 AM INFO:Successfully loaded faiss.
/systems/merlin/systems/dag/ops/feast.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Depre

In [11]:
retrieval = (
    user_features
    >> PredictTensorflow(retrieval_model_path)
    >> QueryFaiss(faiss_index_path, topk=100)
)

2022-03-30 00:40:16.934151: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-30 00:40:18.123955: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0
2022-03-30 00:40:20.257596: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034311152 exceeds 10% of free system memory.


03/30/2022 12:40:21 AM WARNING:No training configuration found in save file, so the model was *not* compiled. Compile it manually.


In [12]:
item_features = retrieval["candidate_ids"] >> QueryFeast.from_feature_view(
    store=feature_store,
    path=feast_repo_path,
    view="item_features",
    column="candidate_ids",
    output_prefix="item",
    include_id=True,
)

user_features_to_unroll = [
    "user_id",
    "user_shops",
    "user_profile",
    "user_group",
    "user_gender",
    "user_age",
    "user_consumption_2",
    "user_is_occupied",
    "user_geography",
    "user_intentions",
    "user_brands",
    "user_categories",
]
combined_features = item_features >> UnrollFeatures(
    "item_id", user_features[user_features_to_unroll]
)

In [13]:
ranking = combined_features >> PredictTensorflow(ranking_model_path)

ordering = combined_features["item_id"] >> SoftmaxSampling(
    relevance_col=ranking["output_1"], topk=10, temperature=20.0
)

2022-03-30 00:40:25.126858: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 788046592 exceeds 10% of free system memory.
2022-03-30 00:40:25.127006: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 788046592 exceeds 10% of free system memory.


The last step of machine learning (ML)/deep learning (DL) pipeline is to deploy the ETL workflow and saved model to production. In the production setting, we want to transform the input data as done during training (ETL). We need to apply the same mean/std for continuous features and use the same categorical mapping to convert the categories to continuous integer before we use the DL model for a prediction. Therefore, we deploy the NVTabular workflow with the Tensorflow model as an ensemble model to Triton Inference using Merlin Systems library very easily. The ensemble model guarantees that the same transformation is applied to the raw inputs.

In [14]:
from merlin.core.dispatch import make_df

export_path = '/Merlin/examples/PoC/poc_ensemble/'

ensemble = Ensemble(ordering, request_schema)
ens_config, node_configs = ensemble.export(export_path)

request = make_df({"user_id": [1]})
request["user_id"] = request["user_id"].astype(np.int32)

In [15]:

response = run_ensemble_on_tritonserver(
    export_path, ensemble.graph.output_schema.column_names, request, "ensemble_model"
)

I0330 00:40:28.996325 1892 tensorflow.cc:2176] TRITONBACKEND_Initialize: tensorflow
I0330 00:40:28.996416 1892 tensorflow.cc:2186] Triton TRITONBACKEND API version: 1.8
I0330 00:40:28.996421 1892 tensorflow.cc:2192] 'tensorflow' TRITONBACKEND API version: 1.8
I0330 00:40:28.996425 1892 tensorflow.cc:2216] backend configuration:
{"cmdline":{"version":"2"}}
I0330 00:40:29.150263 1892 pinned_memory_manager.cc:240] Pinned memory pool is created at '0x7f96b4000000' with size 268435456
I0330 00:40:29.150648 1892 cuda_memory_manager.cc:105] CUDA memory pool is created on device 0 with size 67108864
I0330 00:40:29.156027 1892 model_repository_manager.cc:994] loading: 0_queryfeast:1
I0330 00:40:29.256479 1892 model_repository_manager.cc:994] loading: 1_predicttensorflow:1
I0330 00:40:29.263985 1892 backend.cc:46] TRITONBACKEND_Initialize: nvtabular
I0330 00:40:29.264019 1892 backend.cc:53] Triton TRITONBACKEND API version: 1.8
I0330 00:40:29.264031 1892 backend.cc:56] 'nvtabular' TRITONBACKEND 

Signal (2) received.


I0330 00:40:44.842471 1892 server.cc:252] Waiting for in-flight requests to complete.
I0330 00:40:44.842502 1892 model_repository_manager.cc:1026] unloading: ensemble_model:1
I0330 00:40:44.842628 1892 model_repository_manager.cc:1026] unloading: 6_softmaxsampling:1
I0330 00:40:44.842775 1892 model_repository_manager.cc:1026] unloading: 5_predicttensorflow:1
I0330 00:40:44.842846 1892 model_repository_manager.cc:1132] successfully unloaded 'ensemble_model' version 1
I0330 00:40:44.842869 1892 model_repository_manager.cc:1026] unloading: 4_unrollfeatures:1
I0330 00:40:44.843011 1892 model_repository_manager.cc:1026] unloading: 3_queryfeast:1
I0330 00:40:44.843074 1892 backend.cc:160] TRITONBACKEND_ModelInstanceFinalize: delete instance state
I0330 00:40:44.843092 1892 model_repository_manager.cc:1026] unloading: 2_queryfaiss:1
Signal (I0330 00:40:44.843153 1892 backend.cc:160] TRITONBACKEND_ModelInstanceFinalize: delete instance stateI0330 00:40:44.843172 1892 tensorflow.cc:2363] TRITON

Convert our response to a numpy array and print it out.

In [18]:
output= response.as_numpy('ordered_ids')
output

array([[2628383],
       [1780891],
       [ 397955],
       [2573239],
       [1255680],
       [ 505277],
       [2084603],
       [ 365618],
       [ 229051],
       [1323574]], dtype=int32)

Note that these item ids are encoded values, not the raw original values. We will eventually create the reverse dictionary lookup functionality to be able to map these encoded item ids to their original raw ids with one-line of code. But if you really want to do it now, you can easily map these ids to their original values using the `unique.item_id.parquet` file stored in the `categories` folder.

Well done! You finished deploying a 4-stage Recommender Systems on Triton Inference Server using Merlin framework.